# Pete's workbook

In [21]:
import csv
from deap import gp
from deap import base
import numpy as np
from PIL import Image as im

import IPython.display


# Load the image, vertices, and edges

In [3]:
"""
Setup the inputs
"""

def image2array(img):
    """Convert a bmp image to a numpy array"""    
    return np.reshape(
        np.fromstring(img.tostring(), np.uint8),
        img.size
    )

# Dunno, the orientation of the head I suppose
# but does it need to be an array of 3 arrays?
headpose = np.array([[0],[0],[0]])

# load the photo and just copy it
# well, that's obviously flawed, but we can run with it
imageBmp = im.open('HeadMash.0004.bmp','r')
imageGrey = image2array(imageBmp)
imageRed = image2array(imageBmp)
imageGreen = image2array(imageBmp)
imageBlue = image2array(imageBmp)


In [14]:
"""
Load Vertices
"""
def loadVertices(inFileName):
    ''' Load a csv file and return an array containing the contents of that file
    File format is tab-separated as follows:
    Vertex Number   X   Y   Z
    The file contains a header row.
    Param: string containing filename (and path if not in pwd)
    Returns: a numpy array of the format
    [[<vertex numbers>, ...],
     [<X>, ...             ],
     [<Y>, ...             ],
     [<Z>, ...             ]]
    '''
    inFile = open(inFileName,'r')
    lines = csv.reader(inFile, delimiter = '\t')
    lines.__next__() #skip first row
    vArray = np.array([])
    for line in lines:
        vArray = np.append(vArray,line,1)
    vArray = np.transpose(vArray.reshape(-1,4)) #-1 means as many 
                                                #   along that dimension
    inFile.close()
    return vArray

inputVertices = loadVertices('HeadVertices.txt')
correctVertices = loadVertices('HeadVertices_Morphed1.txt')


In [17]:
"""
Load Edges
"""
def loadEdges(inFileName):
    ''' Load a csv file and return an array containing the contents of that file
    File format is tab-separated as follows:
    EdgeNumber   VertexA   VertexB
    The file contains a header row.
    Param: string containing filename (and path if not in pwd)
    Returns: a numpy array of the format
    [[<edge number>, ...],
     [<vertexA>, ...    ],
     [<vertexB>, ...    ]]
    '''
    inFile = open(inFileName,'r')
    lines = csv.reader(inFile, delimiter = '\t')
    lines.__next__() #skip first row
    vArray = np.array([])
    for line in lines:
        vArray = np.append(vArray,line,0)
    inFile.close()
    vArray = np.transpose(vArray.reshape(-1,3)) #-1 means as many as needed
                                                #   along that dimension
    inFile.close()
    return vArray

inputEdges = loadEdges('HeadEdge.txt')


# Set up the genetic program

In [2]:
"""
Define a population set
    The 'in_types' are:
        boolean (orthographic=True|perspective=False)
        ndarray (head pose 3-1)
        ndarray (grey image 256-256)
        ndarray (red image 256-256)
        ndarray (green image 256-256)
        ndarray (blue image 256-256)
        ndarray (indexed list of vertices 4-n)
        ndarray (indexed list of edges 3-n)
"""
pset = gp.PrimitiveSetTyped(
    name = "main", 
    in_types = [bool, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray], 
    ret_type = np.ndarray
)


In [22]:
"""
Set Up Genetic Programming
"""
toolbox = base.Toolbox()


